# 使用卷积神经网络解决CFIAR-100


> 实际上使用的是VGG-13，但是效果不好；100分类问题训练5轮后accuracy仅为20%

In [1]:
# 导入所需模块
import tensorflow as tf
from tensorflow.keras import datasets,layers,optimizers,metrics,losses,Sequential
import datetime

In [2]:
# 导入数据集
(x,y),(x_test,y_test) = datasets.cifar100.load_data()

In [3]:
y.shape

(50000, 1)

In [4]:
# 数据预处理
'''
1. 归一化
2. 类型转换
3. 整理成tf.Data对象
'''
batch_size = 128

def preprocess(x,y):
    x = tf.cast(x,dtype=tf.float32)/255.
    y = tf.cast(y,dtype=tf.int32)
    y = tf.squeeze(y,axis=0)
    return x,y

db = tf.data.Dataset.from_tensor_slices((x,y))
train_data = db.map(preprocess).shuffle(10000).batch(batch_size)

db_test = tf.data.Dataset.from_tensor_slices((x_test,y_test))
test_data = db_test.map(preprocess).shuffle(10000).batch(batch_size) # test数据可以不用打乱


In [5]:
# 展示
sample = next(iter(train_data))
print(sample[0].shape)
print(sample[1].shape)

(128, 32, 32, 3)
(128,)


In [6]:
# 搭建卷积层、全链接层
conv_net = Sequential([
    layers.Conv2D(64,3,padding = 'same',activation = tf.nn.relu),
    layers.MaxPool2D(strides=2,padding = 'same'),
    
    layers.Conv2D(128,3,padding = 'same',activation = tf.nn.relu),
    layers.MaxPool2D(strides=2,padding = 'same'),
    
    layers.Conv2D(256,3,padding = 'same',activation = tf.nn.relu),
    layers.MaxPool2D(strides=2,padding = 'same'),
    
    layers.Conv2D(512,3,padding = 'same',activation = tf.nn.relu),
    layers.MaxPool2D(strides=2,padding = 'same'),
    
    layers.Conv2D(512,3,padding = 'same',activation = tf.nn.relu),
    layers.MaxPool2D(strides=2,padding = 'same'),
    
    layers.Flatten()
])
conv_net.build(input_shape = [None,32,32,3])
conv_net.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 multiple                  0         
_________________________________________________________________
conv2d_3 (Conv2D)            multiple                  1

In [7]:
# 测试卷积
sample_data = tf.random.normal([10,32,32,3])
out = conv_net(sample_data)
print(out.shape)

(10, 512)


In [8]:
fc_net = Sequential([
    layers.Dense(512,activation=tf.nn.relu),
    layers.Dense(256,activation=tf.nn.relu),
    layers.Dense(100,activation=tf.nn.sigmoid),
])
fc_net.build(input_shape = [None,512])
fc_net.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  262656    
_________________________________________________________________
dense_1 (Dense)              multiple                  131328    
_________________________________________________________________
dense_2 (Dense)              multiple                  25700     
Total params: 419,684
Trainable params: 419,684
Non-trainable params: 0
_________________________________________________________________


In [ ]:
fc_test = tf.random.normal([10,512])
y_fc_test = fc_net(fc_test)
print(y_fc_test.shape)

(10, 100)


In [ ]:
# 搭建训练过程（GradTape实现）
max_iter = 5
optimizer = optimizers.Adam(learning_rate=0.0001)
for iter_step in range(max_iter):
    for i,(ix,iy) in enumerate(train_data):
        begin_time = datetime.datetime.now()
        with tf.GradientTape() as tape:
            out_conv = conv_net(ix)
            out_fc = fc_net(out_conv)
            iy_onehot = tf.one_hot(iy,depth=100)
            ce = losses.categorical_crossentropy(iy_onehot,out_fc)
            loss = tf.reduce_mean(ce)
        grads = tape.gradient(loss,conv_net.trainable_variables + fc_net.trainable_variables)
        optimizer.apply_gradients(zip(grads,conv_net.trainable_variables + fc_net.trainable_variables))
        
        end_time = datetime.datetime.now()
        
        
        if i%50==0:
            print('step {0} : loss = {1} {2} pic/s '.format(i,float(loss),batch_size/(end_time-begin_time).total_seconds()))

#   vaildation
    tot = 0
    for x_t,y_t in test_data:
        out_conv = conv_net(x_t)
        prob = fc_net(out_conv)
        predict = tf.cast(tf.argmax(prob,axis = 1),dtype=tf.int32) 
        corr = tf.equal(y_t,predict)
        corr = tf.reduce_sum(tf.cast(corr,dtype = tf.int32))
        tot += corr
    print('Accuracy = {0}  --> ITER {1}'.format(int(tot)/10000.,iter_step))

W0925 22:31:17.166961 16572 deprecation.py:323] From E:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\ops\math_grad.py:1220: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


step 0 : loss = 4.604790210723877 208.34791497451806 pic/s 
step 50 : loss = 4.594887733459473 2137.679949230101 pic/s 
step 100 : loss = 4.4716973304748535 2333.509561920041 pic/s 
step 150 : loss = 4.364353656768799 2140.3966422527674 pic/s 
step 200 : loss = 4.307084560394287 2292.3457143880514 pic/s 
step 250 : loss = 4.247390270233154 2212.8483507364635 pic/s 
step 300 : loss = 4.239736557006836 2212.7718424783043 pic/s 
step 350 : loss = 3.9944348335266113 2421.4907302307984 pic/s 
